# SNAM

In [1]:
import pandas as pd

In [2]:
# !conda list 

In [3]:
df = pd.read_excel('dataset/Percentuali_di_prelievo_AT_2022-2023.xls')
df = df.rename(columns={"Unnamed: 1": "day_of_week", "Data": "date"})
df['day_of_week'] = pd.to_datetime(df['day_of_week']).dt.day_name()

In [4]:
df.columns

Index(['date', 'day_of_week', 'c1%B1', 'c1%C1', 'c1%D1', 'c1%E1', 'c1%F1',
       'c1%B2', 'c1%C2', 'c1%D2', 'c1%E2', 'c1%F2', 'c1%B3', 'c1%C3', 'c1%D3',
       'c1%E3', 'c1%F3', 'c2%', 'c4%', 't1%1', 't1%2', 't1%3'],
      dtype='object')

C1 Riscaldamento  
C2 Uso cottura cibi e/o produzione di acqua calda  
C3 Riscaldamento + uso cottura cibi e/o produzione acqua calda sanitaria  
C4 Uso condizionamento  
C5 Uso condizionamento + riscaldamento   

T1 Uso tecnologico (artigianale-industriale)  
T2 Uso tecnologico + riscaldamento  

In [5]:
mapping_name_to_type = {
    "c1": "riscaldamento",
    "c2": "cottura+acqua_calda",
    "c3": "riscaldamento + cottura + acqua calda",
    "c4": "condizionamento",
    "c5": "condizionamento + riscaldamento",
    "t1": "uso tecnologico (artigianale-industriale)",
    "t2": "uso tecnologico + riscaldamento"
}

In [6]:
df.columns.str[:2].unique()

Index(['da', 'c1', 'c2', 'c4', 't1'], dtype='object')

Interpretazione del dataset:
- ogni riga è una giornata relativa al consumo di gas
- ogni colonna è una persona/cliente diverso
- il nome della colonna indica il tipo di consumo di quel cliente secondo il mappaggio sopra definito

In [7]:
(
    df.columns[df.columns.str.contains('c1')].shape, 
    df.columns[df.columns.str.contains('c2')].shape, 
    df.columns[df.columns.str.contains('c4')].shape, 
    df.columns[df.columns.str.contains('t1')].shape
)

((15,), (1,), (1,), (3,))

Abbiamo a disposizione:
- 15 clienti a consumo C1 = solo riscaldamento
- 1 cliente a consumo C2 = cottura + acqua calda
- 1 cliente a consumo C4 = condizionamento
- 3 clienti a consumo T1 = uso tecnologico (artigianale-industriale)"

Non è chiaro se questi siano obbligatoriamente persone diverse, o alcuni di questi profili di consumo appartengono allo stesso.  
Si potrebbe provare a controllare se i profili C2, C4, T1 facciano riferimento ai profili C1 di cui disponiamo.  
Per effettuarlo potremmo controllare come si muovono insieme queste serie rispetto a quelle di C1 con una cross-correlazione.

In [13]:
df.drop(columns=['date', 'day_of_week']).sum(axis=0)

c1%B1    100.0
c1%C1    100.0
c1%D1    100.0
c1%E1    100.0
c1%F1    100.0
c1%B2    100.0
c1%C2    100.0
c1%D2    100.0
c1%E2    100.0
c1%F2    100.0
c1%B3    100.0
c1%C3    100.0
c1%D3    100.0
c1%E3    100.0
c1%F3    100.0
c2%      100.0
c4%      100.0
t1%1     100.0
t1%2     100.0
t1%3     100.0
dtype: float64

La il valore j della i-esima colonna, è la percentuale del totale della colonna. Se leggo 0,00010. E' lo 0,00010%.

In [15]:
df['c1%B1'].max()

np.float64(1.0583294467900397)

# Obiettivi analisi descrittiva
0. Quali sono le caratteristiche principali di questo dataset?
1. Quali sono le principali differenze e similitudini fra i diversi profili in termini di stagionalità?
2. Partendo dall’analisi del punto b, a quale tipologia di utilizzo potrebbe realisticamente essere
attribuito ogni profilo?
3. Ci possono essere a tuo parere delle variabili che possono ragionevolmente impattare il
profilo di consumo di un cliente gas che in questi profili non verrebbero considerate?

# Domanda 0

In [11]:
df.describe()

,date,c1%B1,c1%C1,c1%D1,c1%E1,c1%F1,c1%B2,c1%C2,c1%D2,c1%E2,...,c1%B3,c1%C3,c1%D3,c1%E3,c1%F3,c2%,c4%,t1%1,t1%2,t1%3
count,365,3.650000e+02,3.650000e+02,3.650000e+02,3.650000e+02,3.650000e+02,3.650000e+02,3.650000e+02,3.650000e+02,3.650000e+02,...,3.650000e+02,3.650000e+02,3.650000e+02,3.650000e+02,3.650000e+02,365.000000,3.650000e+02,365.000000,3.650000e+02,3.650000e+02
mean,2023-04-01 00:00:00,2.739726e-01,2.739726e-01,2.739726e-01,2.739726e-01,2.739726e-01,2.739726e-01,2.739726e-01,2.739726e-01,2.739726e-01,...,2.739726e-01,2.739726e-01,2.739726e-01,2.739726e-01,2.739726e-01,0.273973,2.739726e-01,0.273973,2.739726e-01,2.739726e-01
min,2022-10-01 00:00:00,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.019865e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,0.158557,1.000000e-08,0.171303,1.000000e-08,1.000000e-08
25%,2022-12-31 00:00:00,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,2.453621e-02,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,0.188991,1.000000e-08,0.272717,2.713328e-01,1.000000e-08
50%,2023-04-01 00:00:00,1.000000e-08,1.000000e-08,1.000000e-08,1.545755e-01,1.573372e-01,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,3.583823e-02,0.237408,1.000000e-08,0.279590,3.263408e-01,3.896684e-01
75%,2023-07-01 00:00:00,6.511353e-01,5.741217e-01,5.598017e-01,5.469917e-01,5.063921e-01,6.245658e-01,6.085329e-01,5.858121e-01,5.605555e-01,...,1.000000e-08,5.911089e-01,5.803681e-01,5.462693e-01,5.317976e-01,0.366369,7.211932e-01,0.289368,3.450471e-01,4.167021e-01
max,2023-09-30 00:00:00,1.058329e+00,9.146862e-01,8.867823e-01,8.247798e-01,7.716074e-01,1.273638e+00,1.068104e+00,1.061207e+00,9.902603e-01,...,1.537172e+00,1.289829e+00,1.282510e+00,1.198137e+00,1.129772e+00,0.440201,9.757592e-01,0.316521,3.860725e-01,4.742298e-01
std,NaN,4.038501e-01,3.459846e-01,3.191266e-01,3.030962e-01,2.700257e-01,4.590206e-01,3.877195e-01,3.666473e-01,3.522351e-01,...,5.186676e-01,4.439024e-01,4.224604e-01,4.068848e-01,3.759001e-01,0.097232,3.962786e-01,0.029027,1.257020e-01,1.884016e-01


In [12]:
fig = px.line(df, x="date", y="c1%B1", )
fig.show()

# Domanda 1

## C1

In [18]:
import plotly.graph_objects as go
# Create traces
fig = go.Figure()

c1_columns = df.columns[df.columns.str.contains('c1')]

for c1_col in c1_columns:
    fig.add_trace(
        go.Scatter(
            x=df['date'], 
            y=df[c1_col],
            mode='lines',
            name=c1_col
        )
    )

fig.show()

I c1 si comportano come profili di riscaldamento partono medi a novembre per poi crescere gradualmente con un picco a gennaio, per poi tornare a scendere verso giugno e luglio.
Hanno diversi comportamenti a livello di intensità. Il c1F3 segue questo andamento. Nota che il valore del riscaldamento ha degli azzeramenti nel corso di quello che è il weekend.
  
Ci sono poi altri che hanno comportamenti più serrati. Come il c1C3 che inizia ad avere un valore rilevante a Novembre, crescere fino a gennaio e poi scendere fino ad azzerarsi a maggio. Nota che il valore del riscaldamento ha degli azzeramenti nel corso di quello che è il weekend.

Abbiamo invece c1F1 che si comporta come desritto per gli altri nel range ottobre, luglio, però non ha gli azzeramenti del weekend.

Si potrebbe immaginare che quegli edifici per cui il riscaldamento si azzera nel weekend si tratti di uffici, magazzini o comunque qualsiasi tipologia di struttura che nel weekend non ha necessità di rimanere riscaldata quando non sono presenti i lavoratori.  
Mentre quelle serie temporali che non hanno il crollo nel weekend si tratta di case.  
Il fatto che non tutti abbiano la zona di crescita, picco, e decrescita nelle stesse date può essere dovuto al fatto che il riscaldamento non è uguale in tutte le zone d'italia, e che le leggi di attivazione del riscaldamento siano di comuni e/o regioni diverse. Per esempio a Milano: "periodo di accensione: dal 22 ottobre 2023 all'8 aprile 2024" [X](https://www.comune.milano.it/aree-tematiche/ambiente/energia/calendario-accensione-impianti)

Di fronte a queste breve considerazioni abbiamo la possibilità di assumere che:
- i profili osservati appartengono a case quando nel weekend non si hanno azzeramenti
- appartengono ad altre strutture inattive nel weekend quando si hanno azzeramenti
- i profili osservati appartengono a zone geografiche diverse di italia

## C2 E C4

In [20]:
import plotly.graph_objects as go
# Create traces
fig = go.Figure()

c1_columns = df.columns[df.columns.str.contains('c2|c4')]

for c1_col in c1_columns:
    fig.add_trace(
        go.Scatter(
            x=df['date'], 
            y=df[c1_col],
            mode='lines',
            name=c1_col
        )
    )

fig.show()

Dalle ricerche online abbiamo ricavato con il c2 è l'indicatore relativo al consumo di acqua calda e gas per cottura. Possiamo osservare come il profilo della serie temporale infatti non si azzeri mai. Inoltre questo cresce nel periodo Ottobre-Gennaio per poi descrescere nel periodo successivo. Questo può essere assunto come la parte di valore relativa al consumo dell'acqua calda, utilizzata maggiormente nella fine del periodo Autunnale e per tutta la prte invernale, per poi descrescere gradualmente in primavera.  
Qusto c2 a causa dei presunti aumenti dovuti all'utilizzo dell'acqua calda, suggerisce come si tratti del consumo di gas di un'abitazione, piuttosto che di un ufficio o altro; anche poiché non presenta il comportamento da ufficio che abbiamo osservato all'interno di c1.

Il c4 dalle ricerche risulta essere il profilo del consumo relativo ai condizionatori. Infatti questo è al minimo per tutta la parte invernale fino al 31 maggio, per poi essere attivato costantemente con qualche decrescita giornaliera, forse nei giorni meno caldi od in cui si è stati meno in casa.